Define cv2_imshow from google collab because it wont install

In [ ]:
import cv2
from IPython import display
import PIL

def cv2_imshow(a):
  """A replacement for cv2.imshow() for use in Jupyter notebooks.

  Args:
    a : np.ndarray. shape (N, M) or (N, M, 1) is an NxM grayscale image. shape
      (N, M, 3) is an NxM BGR color image. shape (N, M, 4) is an NxM BGRA color
      image.
  """
  a = a.clip(0, 255).astype('uint8')
  # cv2 stores colors as BGR; convert to RGB
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display.display(PIL.Image.fromarray(a))


# Configure our Model

using PyTorch download a pre-trained YOLOv5 model for image processing

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.5 #NMS Confidence Threshold
model.multi_label = False
model.classes = [0] # Detect people only
model.max_det = 1000

# Process Images

loop through our directory and see what images have people in them

In [ ]:
import os

directory_path = '/mnt/c/Users/athie/Pictures'
for filename in os.listdir(directory_path):
    if filename.endswith((".jpg", ".jpeg", ".png", ".gif")):

        image_path = os.path.join(directory_path, filename)
        
        # Open and process the image using OpenCV
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB format
       
        # Perform image processing or analysis here
        with torch.no_grad():
            results = model(img)
            results.render()
            dataFrame = results.pandas().xyxy[0]
            
            #print(dataFrame)
            located = len(dataFrame.loc[dataFrame['class'] == 0].index)
            if located:
                print(image_path)
                cv2_imshow(results.ims[0])